# Third submission: Gradient Boosting
 * min observations in each category: 2
 * RMSLE: 1.715
 * best parameters: {'n_iter_no_change': 6, 'n_estimators': 10000, 'min_samples_split': 21, 'min_samples_leaf': 9, 'max_depth': 5, 'learning_rate': 0.06}
 * wasn't able to use xgboost with early stopping and cross validations (used sklearn's version)

In [68]:
%matplotlib inline
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# set seeds
np.random.seed(1)
random.seed(1)

# Clean data

## Create and drop features, manage categorical

In [69]:
# Function to prepare data

from sklearn.base import TransformerMixin

def create_new_features(X):
    # transform utc offset:
    X['Sin_UTC'] = np.sin((11 * 3600 + X['UTC Offset']) / (24 * 3600) * 2 * np.pi)
    X['Cos_UTC'] = np.cos((11 * 3600 + X['UTC Offset']) / (24 * 3600) * 2 * np.pi)
    
    # time since creation in days
    duration = (pd.to_datetime('today') - 
                pd.to_datetime(X['Profile Creation Timestamp']).dt.tz_localize(None))
    X['Duration'] = duration.apply(lambda x: x.days)
    
    # convert personal url into True/False (NaNs or unique)
    X['Has Personal URL'] = X['Personal URL'].notna()
    
    # log(x + 1) of numerical features
    for feature in ['Num of Followers', 'Num of People Following',
                    'Num of Status Updates', 'Num of Direct Messages',
                    'Avg Daily Profile Visit Duration in seconds',
                    'Avg Daily Profile Clicks']:
        X[f'log {feature}'] = np.log1p(X[feature])
    return X
    
def clean_features(X):
    # merge categories names with and without cap letter
    X['Location Public Visibility'] = X['Location Public Visibility'].str.lower()
    X.loc[X['Profile Category'] == " ", 'Profile Category'] = 'unknown'
    return X

def drop_features(X, l_features):
    return X.drop(columns=l_features)
    
class RemoveCategories(TransformerMixin):
    def __init__(self, min_obs=10, l_cols=[]):
        self.min_obs = min_obs
        self.l_cats = {}
        self.l_cols = l_cols

    def fit(self, X):
        # assumes all columns of df_cat are strings
        for col in self.l_cols:
            val_counts = X[col].fillna('missing').value_counts()
            self.l_cats[col] = val_counts[val_counts >= self.min_obs].index.tolist()
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        for col in self.l_cols:
            X.loc[~ X[col].isin(self.l_cats[col]), col] = 'other'        
        return X

In [70]:
# data cleaning and feature engineering

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

l_cat_small = ['Profile Text Color', 'Profile Page Color',
               'Profile Theme Color', 'User Language', 'Location']
l_drop = ['Id', 'User Name', 'Profile Image', 'Personal URL',
          'Profile Creation Timestamp', 'Location', 'UTC Offset']
l_drop_logs = ['Num of Followers', 'Num of People Following',
               'Num of Status Updates', 'Num of Direct Messages',
               'Avg Daily Profile Visit Duration in seconds', 
               'Avg Daily Profile Clicks']

features_to_drop =dict(l_features = l_drop + l_drop_logs)

pipe = Pipeline([('create new features',
                  FunctionTransformer(create_new_features)),
                 ('clean data', FunctionTransformer(clean_features)),
                 ('remove categories', RemoveCategories(min_obs=2, 
                                                        l_cols=l_cat_small)),
                 ('drop variables', FunctionTransformer(drop_features,
                                    kw_args=features_to_drop))
                ])

## Imputation and encoding/scaling

In [71]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# numerical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
    ])

# categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

In [72]:
# train data
df = pd.read_csv('data/train.csv')
y = np.log1p(df['Num of Profile Likes'].values)
X = df.drop(columns='Num of Profile Likes')

X = pipe.fit_transform(X)

preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer,
                   X.select_dtypes(exclude=['object', 'bool']).columns),
                  ('cat', categorical_transformer,
                   X.select_dtypes(include=['object', 'bool']).columns)])

X = preprocessor.fit_transform(X)

# XGBoost

In [77]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor

gb= GradientBoostingRegressor(random_state=0)

parameters = {'n_estimators': [10000],
              'n_iter_no_change': [6, 8, 10],
              'learning_rate': [0.06, 0.07],
              'min_samples_split': range(8, 25),
              'min_samples_leaf': range(3, 10),
              'max_depth': range(3, 7)}

reg_gb = RandomizedSearchCV(gb, parameters, scoring='neg_mean_squared_error',
                            n_jobs=100, random_state=0, n_iter=100, verbose=10)
reg_gb.fit(X, y)

print(f'RMSLE: {np.sqrt(-reg_gb.best_score_):.3f}',
      f'\nbest parameters: { reg_gb.best_params_}')

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=100)]: Using backend LokyBackend with 100 concurrent workers.
[Parallel(n_jobs=100)]: Done  21 tasks      | elapsed:   17.3s
[Parallel(n_jobs=100)]: Done  42 tasks      | elapsed:   19.2s
[Parallel(n_jobs=100)]: Done  65 tasks      | elapsed:   21.8s
[Parallel(n_jobs=100)]: Done  88 tasks      | elapsed:   24.9s
[Parallel(n_jobs=100)]: Done 113 tasks      | elapsed:   34.5s
[Parallel(n_jobs=100)]: Done 138 tasks      | elapsed:   39.0s
[Parallel(n_jobs=100)]: Done 165 tasks      | elapsed:   43.4s
[Parallel(n_jobs=100)]: Done 192 tasks      | elapsed:   48.8s
[Parallel(n_jobs=100)]: Done 221 tasks      | elapsed:   55.1s
[Parallel(n_jobs=100)]: Done 250 tasks      | elapsed:  1.0min
[Parallel(n_jobs=100)]: Done 281 tasks      | elapsed:  1.1min
[Parallel(n_jobs=100)]: Done 352 out of 500 | elapsed:  1.4min remaining:   34.5s
[Parallel(n_jobs=100)]: Done 403 out of 500 | elapsed:  1.6min remaining:   22.8s
[Parallel(n_jobs=100)]: Done 454 out of 500 | elapsed:  1.7min r

RMSLE: 1.715 
best parameters: {'n_iter_no_change': 6, 'n_estimators': 10000, 'min_samples_split': 21, 'min_samples_leaf': 9, 'max_depth': 5, 'learning_rate': 0.06}


# Submission

In [79]:
def create_submission(estimator, number):
    X_test = pd.read_csv('data/test.csv')
    Id = X_test['Id']
    X_test = pipe.transform(X_test)
    X_test = preprocessor.transform(X_test)
    y_pred = (estimator.predict(X_test))
    y_pred_test = pd.Series(np.expm1(y_pred))
    submission = pd.DataFrame({'Id': Id, 'Predicted': y_pred_test})
    path = f'submissions/submission{number}.csv'
    while os.path.exists(path):
        number += 1
        path = f'submissions/submission{number}.csv'
    submission.to_csv(path, index=False)
    print(f'submission #{number} created')

In [80]:
create_submission(reg_gb, 1)

submission #3 created


In [81]:
pd.read_csv('submissions/submission3.csv').head()

,Id,Predicted
0,49I3SOKLI2CMNGP4,3805.131066
1,727IRIR59A3P88LK,2397.606135
2,LN95SD15SRPCEE8F,518.503607
3,TB11I7F0PN033D4T,3660.776378
4,32PSGCK5PATHMR07,198.484124
